# Testing code

# Importing stuff

In [42]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import os.path as osp
import random

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm_notebook

from nlpclass.config import model_config
from nlpclass.data.data_utils import TranslationDataset, text_collate_func
from nlpclass.models.evaluation_utils import bleu_eval, output_to_translations
from nlpclass.models.models import DecoderRNN, EncoderCNN, EncoderRNN, TranslationModel
from nlpclass.models.training_utils import load_data

In [44]:
CURRENT_PATH = os.getcwd()
DATA_DIR = osp.join(CURRENT_PATH, '..', 'data')
MODEL_DIR = osp.join(CURRENT_PATH, '..','models')

# Data

In [168]:
data, data_loaders, max_length = load_data('vi', batch_size=24, subsample=0.025)

Counting words...
Counted words:
eng 6726
vi 7801
Counting words...
Counted words:
eng 3562
vi 3678
Counting words...
Counted words:
eng 3361
vi 3518


In [5]:
encoder = EncoderCNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2).to(model_config.device)
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=128,
    output_size=data['train'].output_lang.n_words,
    attention=False).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [169]:
encoder = EncoderRNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2,
    dropout=0.0,
    bidirectional=True).to(model_config.device)
if encoder.bidirectional:
    multiplier = 2
else:
    multiplier = 1
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=multiplier * 128,
    output_size=data['train'].output_lang.n_words,
    attention=True).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [170]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)

In [171]:
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [172]:
def calc_loss(logits, target, criterion):
    logits_flat = logits.view(-1, logits.size(-1))
    target_flat = target.view(-1, 1).squeeze()
    return criterion(logits_flat, target_flat)

In [173]:
def evaluate(model, data, data_loaders, dataset_type='dev', max_batches=100):
    model.eval()
    with torch.no_grad():
        original_strings = []
        translated_strings = []
        for i, batch in enumerate(data_loaders[dataset_type]):
            if i > max_batches:
                break
            logits = translation_model(batch)
            epoch_loss = calc_loss(logits, batch['target'], criterion)
            original = output_to_translations(batch['target'], data['train'])
            translations = output_to_translations(model.greedy(batch), data['train'])
            original_strings.extend(original)
            translated_strings.extend(translations)
        bleu = bleu_eval(original_strings, translated_strings)
        model.train()
        print(epoch_loss)
        print(bleu)
        
        return original_strings, translated_strings

In [192]:
for i in tqdm_notebook(range(25)):
    for batch in data_loaders['train']:
        translation_model.train()
        optimizer.zero_grad()
        logits = translation_model(batch)
        loss = calc_loss(logits, batch['target'], criterion)
        loss.backward()
        
        encoder_norm = 0
        for p in translation_model.encoder.parameters():
            param_norm = p.grad.data.norm(2)
            encoder_norm += param_norm.item() ** 2
        decoder_norm = 0
        for p in translation_model.decoder.parameters():
            param_norm = p.grad.data.norm(2)
            decoder_norm += param_norm.item() ** 2
            
        print(encoder_norm, decoder_norm, loss.item())
            
        clip_grad_norm_(filter(lambda p: p.requires_grad,
                                   translation_model.parameters()), 5.0)
        
        optimizer.step() 
    original, translation = evaluate(translation_model, data, data_loaders, dataset_type='train')

0.20687949540555664 0.743069744848148 4.013472080230713
0.12416938919414126 0.6604997387830878 3.7681403160095215
0.13508697794634295 0.6325013233520471 3.702760934829712
0.15965794342790685 0.7780846420221936 3.3308780193328857
0.13891497467307007 0.6817508205555265 3.623516082763672
0.18881076632940158 0.6468502333631365 3.6622562408447266
0.14682974048378822 0.789686836431067 3.581509590148926
0.1722922481946444 0.879440958576494 3.5672218799591064
0.1414990225701052 0.8122170260704333 4.425287246704102
0.12318427220892757 0.6475319841955293 3.660111904144287
0.24132099098312035 0.9584002422650365 4.579545021057129
2.613765115920316 2.7143743025408686 3.156601905822754
0.14568374672073903 0.725659107231157 4.8209638595581055
0.15574742905080718 0.7884909870215104 4.6951069831848145
0.15614441856176567 0.7117643127225174 4.787650108337402
0.2310522976092384 0.9225341664906159 3.594244956970215
0.23398980242293765 0.7732889899855478 4.674855709075928
0.2120586559093853 0.9873475229225

KeyboardInterrupt: 

In [193]:
#x = next(iter(data_loaders['train']))
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.greedy(x), data['train'])
print(bleu_eval(original, translations))
print(list(zip(translations[0:3], original[0:3])))

0.6531460812606793
[('in the of that i could be able to you .', 'well in 25 years i was mostly putting out pieces of this romance and not getting a lot back in because design on call doesnt always connect you with a circumstance in which you can produce things of this nature .'), ('these conversations and you and it and and and and and and and and and and and and and tedmed and it and redesign and it and redesign and it and redesign and it and redesign and it and redesign and it .', 'your medical chart its hard to access impossible to read and full of information that could make you healthier if you just knew how to use it . at tedmed thomas goetz looks at medical data making a bold call to redesign it and get more insight from it .'), ('eighteen months in the in the in the in the in the in her congressmen in the united states and the her congressmen her congressmen her congressmen her congressmen in the united states .', 'eighteen months later at the age of 90 she arrived in washingto

In [194]:
translation_model.beam_size = 3

In [195]:
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.beam_search(x), data['train'])
print(bleu_eval(original, translations))
print(list(zip(translations[0:3], original[0:3])))

2.349895955513169
[('in in a lot of the of that .', 'well in 25 years i was mostly putting out pieces of this romance and not getting a lot back in because design on call doesnt always connect you with a circumstance in which you can produce things of this nature .'), ('these conversations and to the it and and and and and and and you and it and goetz and tedmed and tedmed and tedmed .', 'your medical chart its hard to access impossible to read and full of information that could make you healthier if you just knew how to use it . at tedmed thomas goetz looks at medical data making a bold call to redesign it and get more insight from it .'), ('eighteen months in in the united states in the united states in the united states in the united states .', 'eighteen months later at the age of 90 she arrived in washington with hundreds following her including many congressmen who had gotten in a car and driven out about a mile outside of the city to walk in with her .')]


In [116]:
x = next(iter(data_loaders['train']))

optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-2)

for i in range(100):
    optimizer.zero_grad()
    logits = translation_model(x)
    loss = calc_loss(logits, x['target'], criterion)
    print(loss)
    loss.backward()
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(8.9509, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.8092, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.1293, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5016, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6784, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.4387, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6025, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5010, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.3718, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2881, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2512, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.0624, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7472, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9508, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.6674, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.8056, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7100, device='cuda:0', grad_fn=<NllLossBackward

In [29]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [30]:
for i, batch in enumerate(tqdm_notebook(data_loaders['train'])):
    if i % 500 == 0:
        evaluate(translation_model, data, data_loaders)
        #evaluate(translation_model, data, data_loaders, dataset_type='train')
    optimizer.zero_grad()
    logits = translation_model(batch)
    loss = calc_loss(logits, batch['target'], criterion)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(19.1450, device='cuda:0')
0.0008142795832761162
tensor(9.0464, device='cuda:0')
0.0033784249080402885


KeyboardInterrupt: 

In [ ]:
original_strings, translated_strings = evaluate(translation_model, data, data_loaders)

In [13]:
original_strings[0:10]

['baseball be later but of volumes all. hope. answer. only were',
 'be this. 65. know what must truth. i most',
 'games be practical everybody pins your needles all forward careful were',
 'i moment. it attack. be sorrows. ten.',
 'canadian be appointment careful laugh. no truth. something time were',
 'beautiful cupboard. hot. wine these',
 'much. but how bitter really dishonesty fed recently',
 'applied all else fed to climate husbands. matter. key',
 'but they answer. perfect. today. tape',
 'i is invented i situation getting powers your few']

In [14]:
translated_strings[0:10]

['i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the']

In [48]:
decoder_input.size()

torch.Size([16])

In [23]:
total_loss.backward()

In [30]:
predictions = translation_model.greedy(x)

In [215]:
for row in predictions.cpu().numpy():
    decoded_words = []
    for elem in row[1:]:
        decoded_words.append(data['train']['output_lang'].index2word[elem])
        if elem == model_config.EOS_token:
            break

In [167]:
yo = Variable(torch.LongTensor([model_config.SOS_token] * 8)).to(model_config.device)
yo = torch.stack((yo, topi.squeeze(), topi.squeeze()), dim=1)

In [99]:
total_loss.backward()

In [57]:
seq_range = torch.autograd.Variable(torch.LongTensor(np.repeat([2], len(x['input_length'])))).to(model_config.device)
mask = seq_range < x['input_length']
loss = -torch.gather(decoder_output, dim=1, index=input_var.unsqueeze(1)).squeeze() * mask.float()

In [66]:
loss.sum() / torch.sum(loss > 0).float()

tensor(11.4193, device='cuda:0', grad_fn=<DivBackward1>)

In [63]:
torch.sum(loss > 0).cpu().numpy()

array(8)

In [ ]:
encoder_output, encoder_hidden = encoder(x['input'], x['input_length'])

In [290]:
context = None
if decoder.attention:
    context = Variable(torch.zeros(encoder_output.size(0), encoder_output.size(2))).unsqueeze(1).to(model_config.device)

In [291]:
decoder_output, decoder_hidden, context, weights = decoder(input_var, encoder_hidden, encoder_output, context)

In [16]:
def train_model(model, optimizer, train_loader, criterion):
    model.train()
    loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch['label'])
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * \
            len(batch['label']) / len(train_loader.dataset)
    return loss_train

In [17]:
optimizer = torch.optim.Adam(translation_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
train_model(translation_model, optimizer, train_loader, criterion)

KeyError: 'label'